In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from datetime import date, datetime
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_theme()
sns.set_color_codes()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)

# column_names = ['# Seen', 'ALSA', '# Picked', 'ATA', '# GP', 'GP WR', '# OH', 'OH WR', '# GD', 'GD WR', '# GIH', 'GIH WR', '# GND', 'GND WR', 'IWD', 'Color', 'Rarity']

from Utilities import Logger
import WUBRG
from data_graphing.LineColors import LineColors
from game_metadata import SETS, FORMATS
from game_metadata import Card,CardManager, SetMetadata, FormatMetadata
from data_fetching import DataLoader, LoadedData, DataFramer, FramedData, SetManager, CentralManager
from data_fetching.utils.date_helper import get_next_17lands_update_time, get_prev_17lands_update_time

LOG_LEVEL = Logger.FLG.DEFAULT
LOG_LEVEL

<Flags.DEFAULT: 3>

# Initialization

In [3]:
print(f"Last 17Lands Update: {get_prev_17lands_update_time()}")
print(f"Current UTC Time:    {datetime.utcnow()}")
print(f"Next 17Lands Update: {get_next_17lands_update_time()}")

Last 17Lands Update: 2022-04-30 02:00:00
Current UTC Time:    2022-04-30 16:27:04.812922
Next 17Lands Update: 2022-05-01 02:00:00


In [ ]:
data_manager = CentralManager()

In [ ]:
start = datetime.utcnow()
data_manager.check_for_updates()
end = datetime.utcnow()
end - start

In [ ]:
SNC = data_manager['SNC']
NEO = data_manager['NEO']
VOW = data_manager['VOW']
MID = data_manager['MID']
set_data = SNC

In [8]:
set_data = SetManager('SNC')
set_data.check_for_updates()

Checking for missing data for SNC PremierDraft...
Getting overall data for SNC PremierDraft
Updating data for 'CardRatings.json'. Fetching from 17Lands site...
Updating data for 'WCardRatings.json'. Fetching from 17Lands site...
Updating data for 'UCardRatings.json'. Fetching from 17Lands site...
Updating data for 'BCardRatings.json'. Fetching from 17Lands site...
Updating data for 'RCardRatings.json'. Fetching from 17Lands site...


KeyboardInterrupt: 

# Objects

In [ ]:
import sys
try:
    del sys.modules["data_graphing"] 
    del data_graphing
except:
    pass

from data_graphing import ColorHandler, PlotterHelper

In [ ]:
class FramedDataFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._ARCH_FUNCS = ArchFuncs(self._DATA)
        self._SNGL_CARD_FUNCS = SingleCardFuncs(self._DATA)
    
    @property
    def SET(self):
        """The draft set."""
        return self._DATA.SET
    
    @property
    def FULL_SET(self):
        """The full name of the draft set."""
        return self._DATA.FULL_SET
    
    @property
    def FORMAT(self):
        """The format type."""
        return self._DATA.FORMAT
    
    @property
    def SHORT_FORMAT(self):
        """The shorthand of the format type."""
        return self._DATA.FULL_FORMAT
    
    @property
    def DATA(self):
        """The object which contains the data about the set and format."""
        return self._DATA
        
    def check_for_updates(self):
        """Populates and updates all data properties, filling in missing data."""
        self._DATA.check_for_updates()

    def reload_data(self):
        """Populates and updates all data properties, reloading all data."""
        self._DATA.reload_data()  
    
    def deck_group_frame(self, name=None, date=None, summary=False):
        """Returns a subset of the 'GROUPED_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_group_frame(name, date, summary)
     
    def deck_archetype_frame(self, deck_color=None, date=None, summary=False):
        """Returns a subset of the 'SINGLE_ARCHETYPE' data as a DataFrame."""
        return self.DATA.deck_archetype_frame(deck_color, date, summary)
    
    def card_frame(self, name=None, deck_color=None, date=None, card_color=None, card_rarity=None, summary=False):
        """Returns a subset of the 'CARD' data as a DataFrame."""
        return self.DATA.card_frame(name, deck_color, date, card_color, card_rarity, summary)
    
    def compress_date_range_data(self, start_date, end_date, card_name=None):
        """Summarizes card data over a provided set of time."""
        return self.DATA.compress_date_range_data(start_date, end_date, card_name)
        
    
    #ArchFuncs
    def get_games_played(self, deck_color):
        return self._ARCH_FUNCS.get_games_played(deck_color)
        
    def get_avg_winrate(self, day=None, arch='All Decks'):
        return self._ARCH_FUNCS.get_avg_winrate(day, arch)
    
    def get_archetype_frame(self, colors, roll=None):
        return self._ARCH_FUNCS.get_archetype_frame(colors, roll)

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_winrate_history(color_filter, roll)

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_winrate_history(color_filter, roll)       
        
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.get_archetype_playrate_history(color_filter, roll)

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        return self._ARCH_FUNCS.plot_archetype_playrate_history(color_filter, roll)
    
    
    #SingleCardFuncs
    def get_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.get_card_summary(card_name, colors, roll)

    def plot_card_summary(self, card_name, colors='', roll=None):
        return self._SNGL_CARD_FUNCS.plot_card_summary(card_name, colors, roll)
    
    def get_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.get_pick_stats(card_name, roll)
    
    def plot_pick_stats(self, card_name, roll=None):
        return self._SNGL_CARD_FUNCS.plot_pick_stats(card_name, roll)
    
    def card_archetype_performance(self, card_name):
        return self._SNGL_CARD_FUNCS.card_archetype_performance(card_name)
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        return self._SNGL_CARD_FUNCS.stat_archetype_performance(stat_name, color_cols, min_colors, max_colors)

##    # filters = ['exact', 'subset', 'contains', 'adjascent'*]
##    ## TODO: Determine how to handle colourless cards.
##    ## 'exact': 'U' --> 'U'
##    ## 'subset': 'UW' --> 'U', 'W', 'WU'
##    ## 'contains': 'U' --> 'U', 'UW', 'UB', 'UR', 'UG'...
##    ## 'contains': 'UW' --> 'UW', 'UBW', 'URW', 'UGW'...
##    ## 'superset': 'UW' --> 'UW', 'UBW', 'URW', 'UGW'...
##    ## 'adjascent': 'UW' --> 'U', 'W', 'UW', 'UG', 'WG', 'UWG'...
##    def card_color_filter(frame, card_color=None, filter_style=''):
##        if card_color is None: 
##            return frame
##        
##            card_color = WUBRG.get_color_identity(card_color)
##            frame = frame[frame['Color'] == card_color]
##            return frame

    
    def compare_card_evaluations(self, start_date, end_date):
        def inner_func(date):
            df = self.card_frame(date=date, deck_color='')
            df.index = [tup[2] for tup in df.index]
            return df

        first = inner_func(date=start_date)
        last = inner_func(date=end_date)
        diff = last[['ALSA', 'ATA', 'Color', 'Rarity']].copy()
        diff['Δ ALSA'] = first['ALSA'] - last['ALSA']
        diff['Δ ATA'] = first['ATA'] - last['ATA']
        return diff[['ALSA', 'Δ ALSA', 'ATA', 'Δ ATA', 'Color', 'Rarity']]


    def get_top(self, column, count=10, asc=True, card_color=None, card_rarity=None, deck_color='', play_lim=None):
        frame = self.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
        frame = frame.sort_values(column, ascending=asc)

        if card_color is not None:
            card_color = WUBRG.get_color_identity(card_color)
            frame = frame[frame['Color'] == card_color]

        if play_lim is not None:
            if type(play_lim) is float: play_lim *= self.get_games_played(deck_color)
            print(f'Minimum Games played to be included: {play_lim}')
            frame = frame[frame['# GP'] >= play_lim]

        return frame.head(count)

In [ ]:
from data_graphing import ROLL
from WUBRG.consts import COLOR_PAIRS

class ArchFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        
    def get_games_played(self, deck_color):
        if deck_color: return self._DATA.deck_archetype_frame(deck_color=deck_color, summary=True)['Games'].sum()
        else: return self._DATA.deck_group_frame(name='All Decks', summary=True)['Games']
    
    def get_avg_winrate(self, day=None, arch='All Decks'):
        if day: return self._DATA.deck_group_frame(date=day, summary=False).loc[(day, arch)]['Win %']
        else: return self._DATA.deck_group_frame(date=day, summary=True).loc[arch]['Win %']   
    
    def get_archetype_frame(self, colors, roll=None):
        if roll is None: roll = ROLL
        win_rate_frame = self._DATA.deck_archetype_frame(deck_color=colors)
        win_rate_frame.index = [tup[0] for tup in win_rate_frame.index]
        #win_rate_frame = win_rate_frame[['Splash', 'Games', 'Win %']]
        win_rate_frame = win_rate_frame[win_rate_frame['Splash'] == False][['Wins', 'Games']]
        rolling = win_rate_frame.rolling(window=roll, min_periods=1, center=True).mean().round()
        rolling['Win %'] = round((rolling['Wins'] / rolling['Games']) * 100, 2)
        rolling['Avg. Win%'] = [self.get_avg_winrate(idx) for idx in win_rate_frame.index]
        rolling['2C Win%'] = [self.get_avg_winrate(idx, arch='Two-color') for idx in win_rate_frame.index]
        rolling['Win % Offset'] = rolling['Win %'] - rolling['Avg. Win%']
        return rolling

    def get_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
        d = dict()
        for col in COLOR_PAIRS:
            temp_frame = self.get_archetype_frame(col)
            d[col] = temp_frame['Win %']
        d['AVG'] = temp_frame ['Avg. Win%']
        d['2C'] = temp_frame ['2C Win%']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        if color_filter:
            col_filt = [col for col in COLOR_PAIRS if color_filter in col] + ['AVG', '2C'] 
            test_frame = test_frame[col_filt]

        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_archetype_winrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
            
        test_frame = self.get_archetype_winrate_history(color_filter, roll)
        lc = LineColors()
        title = f"Archetype Winrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
            title += f"\n{col_filt}"
        elif roll > 1:
            title += f"\n{rol_filt}"
        
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Win Percent")
        
        
    #TODO: Implement a more generic version of this that takes in a list of deck colours to include as output. 
    def get_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
        d = dict()
        for col in COLOR_PAIRS:
            d[col] = self.get_archetype_frame(col)['Games']

        test_frame = pd.DataFrame.from_dict(d)
        test_frame.index = [idx[5:] for idx in test_frame.index]
        rolling = test_frame.rolling(window=roll, min_periods=1, center=True).mean()
        total = rolling.sum(axis=1)
        playrate = rolling.divide(list(total),axis=0) * 100

        if color_filter:
            col_filt = [col for col in COLOR_PAIRS if color_filter in col]
            playrate = playrate[col_filt]

        return playrate

    def plot_archetype_playrate_history(self, color_filter=None, roll=None):
        if roll is None: roll = ROLL
        test_frame = self.get_archetype_playrate_history(color_filter, roll)
        lc = LineColors()
        title = f"Archetpye Playrates (from 17Lands)\n{self._DATA.SET} - {self._DATA.FORMAT}"

        col_filt = f"Color Filter: {color_filter}"
        rol_filt = f"Rolling Average: {roll} Days"

        if color_filter and roll > 1:
            title += f"\n{col_filt}  -  {rol_filt}"
        elif color_filter:
                title += f"\n{col_filt}"
        elif roll > 1:
                title += f"\n{rol_filt}"
        test_frame.plot(figsize=(20, 10), color=lc.get_col_array(color_filter), title=title, lw=2.5, grid=True)
        plt.xlabel("Date")
        plt.ylabel("Percent of Metagame")

In [ ]:
from data_graphing import ROLL
from WUBRG import COLOR_COMBINATIONS
from WUBRG.consts import COLOR_PAIRS


class SingleCardFuncs:   
    def __init__(self, DATA):
        self._DATA = DATA
        self._COLOR_IDX = 0
        
    def _shorten_data(self, card_name, roll, cols, colors=''):
        frame = self._DATA.card_frame(name=card_name, deck_color=colors)[cols]
        frame.index = [tup[0][5:] for tup in frame.index]
        rolling = frame.rolling(window=roll, min_periods=1, center=True).mean()
        return rolling

    def plot_card_summary(self, card_name, colors='', roll=None):
        if roll is None: roll = ROLL        
        rolling = self._shorten_data(card_name, roll, ['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen'], colors=colors)
        
        color_dict = {
            'GIH WR' : (0.33, 0.66, 0.41, 0.9),
            'GND WR' : (0.77, 0.31, 0.32, 0.9),
            'ATA' : (0.87, 0.52, 0.32, 0.9),
            'ALSA' : (0.3, 0.45, 0.69, 0.9),
            '# GP' : (0.51, 0.45, 0.7, 0.9),
            '# GIH' : (0.33, 0.66, 0.41, 0.9),
            '# Picked' : (0.87, 0.52, 0.32, 0.9),
            '# Seen' : (0.3, 0.45, 0.69, 0.9)
        }
        
        plot_help = PlotterHelper(self._DATA, color_dict=color_dict)
        fig, ax = plot_help.new_quad_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(colors=colors, roll=roll)
                
        plot_help.set_labels(y_label="Win Percent", g_x=0, g_y=0)
        plot_help.set_data(rolling, ['GIH WR', 'GND WR'], g_x=0, g_y=0)
        
        plot_help.set_labels(y_label="Pick Number", g_x=0, g_y=1)
        plot_help.set_data(rolling, ['ALSA', 'ATA'], inv_y=True, g_x=0, g_y=1)
        
        plot_help.set_labels(x_label="Date", y_label="# of Games", g_x=1, g_y=0)
        plot_help.set_data(rolling, ['# GP', '# GIH'], g_x=1, g_y=0)

        plot_help.set_labels(x_label="Date", y_label="# of Cards", g_x=1, g_y=1)
        plot_help.set_data(rolling, ['# Seen', '# Picked'], g_x=1, g_y=1)
        
        
        plot_help.save_fig(f"pcs_{card_name}_{colors}.png")
    
    
    def plot_pick_stats(self, card_name, roll=None):
        if roll is None: roll = ROLL        
        taken_data = self._shorten_data(card_name, roll, ['ALSA', 'ATA'])
        
        plot_help = PlotterHelper(self._DATA)
        fig, ax = plot_help.new_single_plot(card_name)
        plot_help.accredit()
        plot_help.desc_note(roll=roll)
        
        plot_help.set_labels(x_label="Date", y_label="Pick Number")
        plot_help.set_data(taken_data, ['ALSA', 'ATA'], inv_y=True)
        
        plot_help.save_fig(f"pps_{card_name}.png")


    def card_archetype_performance(self, card_name):
        d = dict()
        d['AVG'] = self._DATA.card_frame(card_name, '', summary=True)
        for col in COLOR_PAIRS:
            d[col] = self._DATA.card_frame(card_name, col, summary=True)

        test_frame = pd.DataFrame.from_dict(d).T
        return test_frame
    
    
    def stat_archetype_performance(self, stat_name, color_cols=None, min_colors=0, max_colors=5):
        series = self._DATA.card_frame(summary=True)[stat_name]
        frame = series.reset_index(level=0)
        ret = pd.pivot_table(frame, index='Name', columns='Deck Colors')
        ret.columns = ret.columns.droplevel(0)
        ret = ret[COLOR_COMBINATIONS]  #Re-orders the columns in WUBRG order
        if color_cols is not None:
            ret = ret[color_cols]
        return ret

# Examine Data

In [7]:
set_data.BO1.DATA.CARD_HISTORY_FRAME

# Seen  ALSA  # Picked  \
Date       Deck Colors Name                                               
2022-04-28             Angelic Observer             660 4.789        81   
                       Backup Agent                1965 5.346       262   
                       Ballroom Brawlers            509 3.605       108   
                       Boon of Safety              2342 6.952       216   
                       Brokers Initiate            2206 7.150       215   
...                                                 ...   ...       ...   
2022-04-29 WUBRG       Spara's Headquarters         544 1.768       235   
                       Tramway Station            13230 5.345      2697   
                       Waterfront District        13113 5.389      2739   
                       Xander's Lounge              554 1.930       252   
                       Ziatora's Proving Ground     536 1.789       230   

                                                   ATA  # GP  GP WR  # OH  \
Date       Deck Colors Name                                                 
2022-04-28             Angelic Observer          6.531   118 53.390    12   
                       Backup Agent              7.714   597 58.794   106   
                       Ballroom Brawlers         4.139   179 49.162    22   
                       Boon of Safety           10.440   208 53.846    29   
                       Brokers Initiate         10.781   185 54.054    41   
...                                                ...   ...    ...   ...   
2022-04-29 WUBRG       Spara's Headquarters      1.885    43 37.209     2   
                       Tramway Station           7.286   343 49.563    53   
                       Waterfront District       7.396   317 47.319    64   
                       Xander's Lounge           2.024    65 47.692    18   
                       Ziatora's Proving Ground  1.887    68 51.471    10   

                                                 OH WR  # GD  GD WR  # GIH  \
Date       Deck Colors Name                                                  
2022-04-28             Angelic Observer         58.333    38 57.895     50   
                       Backup Agent             60.377   169 62.130    275   
                       Ballroom Brawlers        50.000    47 55.319     69   
                       Boon of Safety           51.724    50 58.000     79   
                       Brokers Initiate         56.098    41 58.537     82   
...                                                ...   ...    ...    ...   
2022-04-29 WUBRG       Spara's Headquarters     50.000    14 42.857     16   
                       Tramway Station          56.604   105 57.143    158   
                       Waterfront District      43.750    80 46.250    144   
                       Xander's Lounge          55.556    14 71.429     32   
                       Ziatora's Proving Ground 60.000    19 68.421     29   

                                                GIH WR  # GND GND WR    IWD  \
Date       Deck Colors Name                                                   
2022-04-28             Angelic Observer         58.000     70 50.000  8.000   
                       Backup Agent             61.455    322 56.522  4.933   
                       Ballroom Brawlers        53.623    110 46.364  7.260   
                       Boon of Safety           55.696    129 52.713  2.983   
                       Brokers Initiate         57.317    103 51.456  5.861   
...                                                ...    ...    ...    ...   
2022-04-29 WUBRG       Spara's Headquarters     43.750     27 33.333 10.417   
                       Tramway Station          56.962    185 43.243 13.719   
                       Waterfront District      45.139    173 49.133 -3.994   
                       Xander's Lounge          62.500     33 33.333 29.167   
                       Ziatora's Proving Ground 65.517     39 41.026 24.492   

                            

In [5]:
print(type(set_data.BO1.DATA.CARD_HISTORY_FRAME))
set_data.BO1.DATA.CARD_HISTORY_FRAME.loc['2022-04-28']

<class 'pandas.core.frame.DataFrame'>


# Seen  ALSA  # Picked    ATA  # GP  \
Deck Colors Name                                                            
            Angelic Observer             660 4.789        81  6.531   118   
            Backup Agent                1965 5.346       262  7.714   597   
            Ballroom Brawlers            509 3.605       108  4.139   179   
            Boon of Safety              2342 6.952       216 10.440   208   
            Brokers Initiate            2206 7.150       215 10.781   185   
...                                      ...   ...       ...    ...   ...   
WUBRG       Spara's Headquarters          71 1.563        26  1.923     1   
            Tramway Station             2101 5.358       486  7.115    19   
            Waterfront District         2014 5.358       414  6.932    17   
            Xander's Lounge               76 2.000        32  2.062     4   
            Ziatora's Proving Ground      73 1.877        27  1.741     5   

                                       GP WR  # OH  OH WR  # GD   GD WR  \
Deck Colors Name                                                          
            Angelic Observer          53.390    12 58.333    38  57.895   
            Backup Agent              58.794   106 60.377   169  62.130   
            Ballroom Brawlers         49.162    22 50.000    47  55.319   
            Boon of Safety            53.846    29 51.724    50  58.000   
            Brokers Initiate          54.054    41 56.098    41  58.537   
...                                      ...   ...    ...   ...     ...   
WUBRG       Spara's Headquarters     100.000     0    NaN     1 100.000   
            Tramway Station           42.105     3  0.000     4  75.000   
            Waterfront District       35.294     2 50.000    10  40.000   
            Xander's Lounge           25.000     0    NaN     1   0.000   
            Ziatora's Proving Ground  40.000     2  0.000     2  50.000   

                                      # GIH  GIH WR  # GND  GND WR     IWD  \
Deck Colors Name                                                             
            Angelic Observer             50  58.000     70  50.000   8.000   
            Backup Agent                275  61.455    322  56.522   4.933   
            Ballroom Brawlers            69  53.623    110  46.364   7.260   
            Boon of Safety               79  55.696    129  52.713   2.983   
            Brokers Initiate             82  57.317    103  51.456   5.861   
...                                     ...     ...    ...     ...     ...   
WUBRG       Spara's Headquarters          1 100.000      0     NaN   0.000   
            Tramway Station               7  42.857     12  41.667   1.190   
            Waterfront District          12  41.667      5  20.000  21.667   
            Xander's Lounge               1   0.000      3  33.333 -33.333   
            Ziatora's Proving Ground      4  25.000      1 100.000 -75.000   

                                     Color Rarity  
Deck Colors Name                                   
            Angelic Observer             W      U  
            Backup Agent                 W      C  
            Ballroom Brawlers            W      U  
            Boon of Safety               W      C  
            Brokers Initiate           WUG      C  
...                                    ...    ...  
WUBRG       Spara's Headquarters       WUG      R  
            Tramway Station             BR      C  
            Waterfront District         UB      C  
            Xander's Lounge            UBR      R  
            Ziatora's Proving Ground   BRG      R  

[8352 rows x 17 columns]

In [ ]:
examiner = FramedDataFuncs(set_data.BO1)

In [ ]:
examiner.plot_card_summary('Girder Goons')

In [ ]:
examiner._SNGL_CARD_FUNCS._shorten_data('Girder Goons', 3, ['GIH WR', 'GND WR', 'ATA', 'ALSA', '# GP', '# GIH', '# Picked', '# Seen'])

In [ ]:
examiner.plot_pick_stats('Girder Goons')

In [ ]:
examiner.card_archetype_performance('Girder Goons')

In [ ]:
examiner.stat_archetype_performance('GIH WR')

In [ ]:
raise Exception('Stopping Auto-Run!')

## Current Tests

In [ ]:
def gt(x, y):  #Used for function pointer shenanigans
    return x > y

def lt(x, y):  #Used for function pointer shenanigans
    return x < y

def filter_quadrant_cards_df(func1, func2, iwd_thresh=0, play_lim=0.01, card_rarity=None, deck_color=''):
    frame = set_data.BO1.card_frame(deck_color=deck_color, summary=True, card_rarity=card_rarity)
    
    # TODO: Make the mean different based on card rarity so cards aren't moved into incorrect categories.
    
    if play_lim is not None:
        if type(play_lim) is float: play_lim *= set_data.BO1.get_games_played(deck_color)
        print(f'Minimum Games played to be included: {play_lim}')
        frame = frame[frame['# GP'] >= play_lim]
        
    games_played_mean = frame['# GP'].mean()
    cards = frame[func1(frame['IWD'], iwd_thresh)]
    cards = cards[func2(cards['# GP'], games_played_mean)]
    cards = cards.sort_values('IWD', ascending=func1==lt)
    return cards


def get_trap_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_niche_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_staple_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(gt, gt, iwd_thresh, play_lim, card_rarity, deck_color)

def get_dreg_cards(card_rarity=None, deck_color='', iwd_thresh=0, play_lim=0.01):
    return filter_quadrant_cards_df(lt, lt, iwd_thresh, play_lim, card_rarity, deck_color)

In [ ]:
get_niche_cards(deck_color='GW', card_rarity='CU')

In [ ]:
set_data.BO1.card_archetype_performance("Era of Enlightenment")

# TODO

- Calculate archetype openess
 - GIH WR & ALSA based
 - 2.25 of a common per draft 
- Improve graphing capabilities
 - Modify Graphs so they also save an image to a cache folder.
 - Implement better graph titles and axes
 - Update functions to be more general, and have DataFrames piped into them.
- Move sets of files into subfolders based on functionality and relationship (DataFetch, Cards, Graphing, Utils, etc.)
- Better group settings and consts into one file/location.

# Data Graphing and Display

## Card Summary

In [ ]:
set_data.BO1.plot_card_summary('Imperial Oath')

In [ ]:
set_data.BO1.plot_card_summary("Befriending the Moths", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("Michiko's Reign of Truth", roll=3)

In [ ]:
set_data.BO1.plot_card_summary("The Fall of Lord Konda", roll=3)

In [ ]:
set_data.BO1.plot_card_summary('Sunblade Samurai', roll=3)

In [ ]:
set_data.BO1.get_top('GIH WR', count=25, asc=False, deck_color='', card_color=None, card_rarity='RM', play_lim=0.005)

## Card Pick Order Stats

In [ ]:
set_data.BO1.plot_pick_stats('Imperial Oath')

In [ ]:
set_data.BO1.plot_pick_stats('Behold the Unspeakable', 3)

In [ ]:
to_graph = ['Imperial Oath', 'Behold the Unspeakable', 'Virus Beetle', 'Network Disruptor', 'Kumano Faces Kakkazan', 'Iron Apprentice']
for card in to_graph:
    set_data.BO1.plot_pick_stats(card, 3)

## Archetype Winrate History

In [ ]:
set_data.BO1.get_archetype_winrate_history()

In [ ]:
set_data.BO1.plot_archetype_winrate_history('')

In [ ]:
set_data.BO3.plot_archetype_winrate_history('')

## Archetype Playrate History

In [ ]:
set_data.BO1.get_archetype_playrate_history()

In [ ]:
set_data.BO1.plot_archetype_playrate_history('')

In [ ]:
set_data.BO3.plot_archetype_playrate_history('')

## Card Pick Order Changes

In [ ]:
diff = set_data.BO1.compare_card_evaluations('2022-02-10', '2022-03-27')
commons = diff[diff['Rarity'] == 'C']
uncommons = diff[diff['Rarity'] == 'U']

In [ ]:
SRT_TRG = 'Δ ATA'
commons.sort_values(SRT_TRG, ascending=False).head(20)

In [ ]:
commons.sort_values(SRT_TRG, ascending=True).head(20)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=False).head(10)

In [ ]:
uncommons.sort_values(SRT_TRG, ascending=True).head(10)